In [56]:
import pandas as pd
import numpy as np
import ast
import networkx as nx
import matplotlib

In [57]:
df = pd.read_csv('final_papers_dataset.csv', usecols = ["id","author_ids"])

df['author_ids'] = df['author_ids'].apply(ast.literal_eval) # convert string to list

### Create List of Collaborations between authors

In [58]:
collabs = []
for work_idx in range(len(df["id"])):
    work = df["id"][work_idx]
    collabs.extend([(a, b) for idx, a in enumerate(df["author_ids"][work_idx]) for b in df["author_ids"][work_idx][idx + 1:]])
    if work_idx == 100:
        break


### Count the number of occurences of the Collaborations

In [59]:
authors_collabs = []
for authors in collabs:
    count = 0
    for j in range(len(collabs)):
        if authors == collabs[j]:
            count += 1
    authors_collabs.append((authors[0],authors[1],count))


In [85]:

G = nx.Graph()
G.add_weighted_edges_from(authors_collabs)
G.edges(data=True)

G.nodes['https://openalex.org/A5075442335']


{}

### Get unique others in the collaborations

In [61]:
authors = []
for i in range(len(collabs)):
    if collabs[i][0] not in authors:
        authors.append(collabs[i][0])


### Collect data for the authors

In [89]:
import requests
from datetime import datetime

G = nx.Graph()
G.add_weighted_edges_from(authors_collabs)
G.edges(data=True)

for i in range(1):
    BASE_URL = "https://api.openalex.org/authors"
    SEARCH = f"/{authors[i]}"
    SELECT = "?select=id,display_name,works_api_url,cited_by_count,last_known_institutions"

    req = requests.get(BASE_URL + SEARCH + SELECT)

    result = req.json()
    country = result['last_known_institutions'][0]['country_code']
    name = result['display_name']
    cites = result['cited_by_count']
    id = result['id']
    req2 = requests.get(result["works_api_url"])

    authors_work = req2.json()
    date = datetime.strptime("3000-01-01", "%Y-%m-%d")
    # for j in range(len(authors_work['results'])):
    for j in range(len(authors_work['results'])):
        field = authors_work['results'][j]['primary_topic']['field']['display_name']
        if field == "Sociology" or "Psychology" or "Economics" or "Political Science":
            date_string = authors_work['results'][j]['created_date']
            if date > datetime.strptime(date_string, "%Y-%m-%d"):
                date = datetime.strptime(date_string, "%Y-%m-%d")
                
    year_firstpub = date.date().year
    author_attribute = {'display_name': name, 'country': country, 'citation_count': cites, 'first_publication' : year_firstpub}
    nx.set_node_attributes(G, {id : author_attribute})
    print(id)

https://openalex.org/A5075442335


{'display_name': 'Lei Zhou',
 'country': 'CN',
 'citation_count': 7154,
 'first_publication': 2016}